In [1]:
import os
import json

import openai
from openai import OpenAI

In [2]:
api_key = os.getenv("OPENAI_API_KEY_MA")
if api_key is None:
    raise ValueError(
        "API key not found. Please set the OPENAI_API_KEY environment variable."
    )

In [3]:
oai_client = OpenAI(api_key=api_key)
gpt_model = "gpt-4o-mini-2024-07-18"

In [9]:
def rephrase_text(text_to_rephrase, client, model, language="en", number_of_rephrasings=3):
    supported_languages = [
        "en",
        "de",
        "es",
        "fr",
        "it",
        "ko",
        "pt",
        "ru",
        "zh",
    ]

    if language not in supported_languages:
        raise ValueError(
            f"Language '{language}' is not supported. Supported languages are: {supported_languages}"
        )
    
    # Validate number_of_rephrasings
    if not isinstance(number_of_rephrasings, int) or number_of_rephrasings < 1:
        raise ValueError("number_of_rephrasings must be a positive integer")
    
    # Dictionary of prompts for different languages
    prompts = {
        "en": {
            "system": f"You are an expert in paraphrasing text. Your task is to generate {number_of_rephrasings} distinct rephrasings of the provided text while maintaining the original meaning. Ensure that each rephrasing is unique and does not simply repeat the original text.",
            "user": f"Generate {number_of_rephrasings} paraphrases of the following text:\n\n{text_to_rephrase}\n\n"
                   f"Do not divert from the original meaning in any way. Use different words and sentence structures to convey the same message.\n"
                   f"Make sure to provide {number_of_rephrasings} distinct rephrasings."
        },
        "de": {
            "system": f"Du bist ein Experte für das Paraphrasieren von Texten. Deine Aufgabe ist es, {number_of_rephrasings} verschiedene Umschreibungen des bereitgestellten Textes zu erstellen, während die ursprüngliche Bedeutung beibehalten wird. Stelle sicher, dass jede Umschreibung einzigartig ist und den Originaltext nicht einfach wiederholt.",
            "user": f"Erzeuge {number_of_rephrasings} Umschreibungen des folgenden Textes:\n\n{text_to_rephrase}\n\n"
                   f"Weiche in keiner Weise von der ursprünglichen Bedeutung ab. Verwende andere Wörter und Satzstrukturen, um dieselbe Botschaft zu vermitteln.\n"
                   f"Stelle sicher, dass du {number_of_rephrasings} verschiedene Umschreibungen angibst."
        },
        "es": {
            "system": f"Eres un experto en parafrasear textos. Tu tarea es generar {number_of_rephrasings} reformulaciones distintas del texto proporcionado, manteniendo el significado original. Asegúrate de que cada reformulación sea única y no repita simplemente el texto original.",
            "user": f"Genera {number_of_rephrasings} paráfrasis del siguiente texto:\n\n{text_to_rephrase}\n\n"
                   f"No te desvíes del significado original de ninguna manera. Usa diferentes palabras y estructuras de oraciones para transmitir el mismo mensaje.\n"
                   f"Asegúrate de proporcionar {number_of_rephrasings} reformulaciones distintas."
        },
        "fr": {
            "system": f"Vous êtes un expert en reformulation de texte. Votre tâche consiste à générer {number_of_rephrasings} reformulations distinctes du texte fourni tout en maintenant le sens original. Assurez-vous que chaque reformulation est unique et ne répète pas simplement le texte original.",
            "user": f"Générez {number_of_rephrasings} reformulations du texte suivant:\n\n{text_to_rephrase}\n\n"
                   f"Ne vous écartez en aucun cas du sens original. Utilisez des mots et des structures de phrases différents pour transmettre le même message.\n"
                   f"Assurez-vous de fournir {number_of_rephrasings} reformulations distinctes."
        },
        "it": {
            "system": f"Sei un esperto nella parafrasi di testi. Il tuo compito è generare {number_of_rephrasings} riformulazioni distinte del testo fornito mantenendo il significato originale. Assicurati che ogni riformulazione sia unica e non ripeta semplicemente il testo originale.",
            "user": f"Genera {number_of_rephrasings} parafrasi del seguente testo:\n\n{text_to_rephrase}\n\n"
                   f"Non deviare in alcun modo dal significato originale. Usa parole e strutture di frase diverse per trasmettere lo stesso messaggio.\n"
                   f"Assicurati di fornire {number_of_rephrasings} riformulazioni distinte."
        },
        "ko": {
            "system": f"당신은 텍스트를 바꾸는 전문가입니다. 당신의 임무는 제공된 텍스트의 원래 의미를 유지하면서 {number_of_rephrasings}가지 독특한 바꾸기를 생성하는 것입니다. 각 바꾸기가 독특하고 원본 텍스트를 단순히 반복하지 않도록 하십시오.",
            "user": f"다음 텍스트의 {number_of_rephrasings}가지 바꾸기를 생성하십시오:\n\n{text_to_rephrase}\n\n"
                   f"어떤 식으로든 원래 의미에서 벗어나지 마십시오. 동일한 메시지를 전달하기 위해 다른 단어와 문장 구조를 사용하십시오.\n"
                   f"{number_of_rephrasings}가지 독특한 바꾸기를 제공해야 합니다."
        },
        "pt": {
            "system": f"Você é um especialista em parafrasear textos. Sua tarefa é gerar {number_of_rephrasings} reformulações distintas do texto fornecido, mantendo o significado original. Certifique-se de que cada reformulação seja única e não repita simplesmente o texto original.",
            "user": f"Gere {number_of_rephrasings} paráfrases do seguinte texto:\n\n{text_to_rephrase}\n\n"
                   f"Não se desvie do significado original de forma alguma. Use palavras e estruturas de frases diferentes para transmitir a mesma mensagem.\n"
                   f"Certifique-se de fornecer {number_of_rephrasings} reformulações distintas."
        },
        "ru": {
            "system": f"Вы являетесь экспертом в перефразировании текста. Ваша задача - создать {number_of_rephrasings} различных перефразировки предоставленного текста, сохраняя оригинальное значение. Убедитесь, что каждая перефразировка уникальна и не просто повторяет оригинальный текст.",
            "user": f"Создайте {number_of_rephrasings} перефразировки следующего текста:\n\n{text_to_rephrase}\n\n"
                   f"Ни в коем случае не отклоняйтесь от оригинального смысла. Используйте разные слова и структуры предложений, чтобы передать то же сообщение.\n"
                   f"Убедитесь, что вы предоставили {number_of_rephrasings} различных перефразировки."
        },
        "zh": {
            "system": f"你是一个文本改写的专家。你的任务是生成提供的文本的{number_of_rephrasings}种不同的改写，同时保持原意。确保每个改写都是独特的，而不是简单地重复原始文本。",
            "user": f"生成以下文本的{number_of_rephrasings}种改写：\n\n{text_to_rephrase}\n\n"
                   f"无论如何都不要偏离原意。使用不同的单词和句子结构来传达相同的信息。\n"
                   f"确保提供{number_of_rephrasings}种不同的改写。"
        }
    }

    try:
        response = client.responses.create(
            model=model,
            input=[
                {
                    "role": "system",
                    "content": prompts[language]["system"],
                },
                {
                    "role": "user",
                    "content": prompts[language]["user"],
                },
            ],
            text={
                "format": {
                    "type": "json_schema",
                    "name": "rephrasings_list",
                    "strict": True,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "rephrasings": {
                                "type": "array",
                                "description": f"A list of {number_of_rephrasings} rephrased versions of the original input.",
                                "items": {
                                    "type": "string",
                                    "description": "A single rephrased sentence.",
                                },
                            },
                        },
                        "required": ["rephrasings"],
                        "additionalProperties": False,
                    },
                }
            },
        )
        try:
            # Attempt to parse the response as JSON
            event = json.loads(response.output_text)
            return event
        except json.JSONDecodeError as json_error:
            # Handle JSON parsing errors
            print(f"JSONDecodeError: {json_error}")
            return {"error": "Invalid JSON response from API"}
    except openai.BadRequestError as e:
        error_message = str(e)  # Extract the error message as a string
        print(f"Error: {error_message}")
        return {"error": error_message}

In [17]:
sentence = {
    "en": "The quick brown fox jumps over the lazy dog",
    "de": "Der schnelle braune Fuchs springt über den faulen Hund",
    "es": "El rápido zorro marrón salta sobre el perro perezoso",
    "fr": "Le renard brun rapide saute par-dessus le chien paresseux",
    "it": "La veloce volpe marrone salta sopra il cane pigro",
    "ko": "빠른 갈색 여우가 게으른 개를 뛰어넘습니다",
    "pt": "A rápida raposa marrom pula sobre o cão preguiçoso",
    "ru": "Быстрая коричневая лиса прыгает через ленивую собаку",
    "zh": "快速的棕色狐狸跳过懒狗"
}

In [30]:
choice = "zh"

rephrase = rephrase_text(
    sentence[choice],
    client=oai_client,
    model=gpt_model,
    language=choice,
    number_of_rephrasings=7,
)

In [31]:
rephrase

{'rephrasings': ['一只迅速的棕色狐狸跳过一只懒惰的狗。',
  '棕色的狐狸以飞快的速度跃过那只慵懒的狗。',
  '快速的棕色狐狸越过一条懒洋洋的狗。',
  '轻快的棕色狐狸跳过一只不思进取的狗。',
  '一只棕色狐狸快速地跃过那只典型的懒狗。',
  '棕色狐狸以迅速的姿态跳过了一只懒惰的狗。',
  '动作敏捷的棕色狐狸越过了那只松松垮垮的狗。']}

In [32]:
for phrasing in rephrase["rephrasings"]:
    print(phrasing)

一只迅速的棕色狐狸跳过一只懒惰的狗。
棕色的狐狸以飞快的速度跃过那只慵懒的狗。
快速的棕色狐狸越过一条懒洋洋的狗。
轻快的棕色狐狸跳过一只不思进取的狗。
一只棕色狐狸快速地跃过那只典型的懒狗。
棕色狐狸以迅速的姿态跳过了一只懒惰的狗。
动作敏捷的棕色狐狸越过了那只松松垮垮的狗。
